---

# **Pipeline de Dados** | Telegram

Este é um projeto desenvolvido com objetivos de estudo, desenvolvido durante o curso 'Analista de Dados'.

Aluno [Tobias Melo](https://www.linkedin.com/in/tobias-melow/)<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---



<h1><strong>Tópicos</strong></h1>

<ol type="1">
  <li><a href="#introducao">Introdução</a>;</li>
  <li><a href="#arquitetura">Arquitetura</a>;</li>
  <li><a href="#dados">Dados;</a></li>
  <li><a href="#analise">Análise Exploratória de Dados</a>.</li>
</ol>


<h1 id="introducao"><strong>1. Introdução</strong></h1>

Neste projeto desenvolveremos uma aplicação capaz de ingerir informações de um grupo do **Telegram** através de um **chatbot**, essa aplicação contará com um **pipeline de dados** completo.

O **Telegram** permite que as mensagens enviadas ao *bot* podem ser capturadas até mesmo no privado, mas restringimos a opção e utilizaremos o *bot* em um grupo.

<h3><strong>1.1. Chatbot </h3></strong>

> Um **chatbot** é um programa de computador projetado para simular uma conversa com humanos, geralmente através de meios de comunicação baseados em texto ou voz. Eles são usados ​​em uma variedade de aplicações, como atendimento ao cliente, comércio eletrônico, entretenimento e muito mais.

* Exemplos de **chatbots** incluem:
    * Um assistente de atendimento ao cliente que pode responder perguntas comuns sobre uma empresa e ajudar a encaminhar clientes para o departamento apropriado.
    * Um assistente virtual para um comércio eletrônico que pode recomendar produtos e ajudar os clientes a navegar pelo site.
    * Um chatbot de entretenimento que pode contar piadas ou responder perguntas sobre celebridades ou eventos esportivos.

<h3><strong>1.2. Telegram </h3></strong>

> **Telegram é uma plataforma de mensagens instantâneas** baseada na nuvem que permite aos usuários enviar mensagens de texto, fotos, vídeos e arquivos de maneira segura e rápida. Ele foi fundado em 2013 por Pavel Durov, o criador da popular rede social russa VKontakte. Telegram tem sido elogiado por sua segurança e privacidade, e rapidamente ganhou milhões de usuários em todo o mundo. É muito popular entre desenvolvedores por ser pioneiro na implantação da funcionalidade de criação de **chatbots**, que, por sua vez, permitem a criação de diversas automações.

<h3><strong>1.3. AWS </h3></strong>

> **Amazon Web Services (AWS)** é um conjunto de serviços de nuvem empresarial oferecido pela Amazon. Ele oferece uma variedade de serviços, incluindo computação em nuvem, armazenamento, banco de dados, análise e muito mais. Ele permite que as empresas escalem seus recursos de acordo com as necessidades do negócio e pagam apenas pelo que usam. Neste projeto a AWS será responsável por todo o mapeamento dos processos, desde a captação dos dados até a entrega final.

<h1 id="arquitetura"><strong>2. Arquitetura</strong></h1>

Neste tópico, evidenciarei como será toda a estrutura do projeto. Vamos construir um pipeline de dados que ingira, processe, armazene e exponha mensagens de um grupo do **Telegram** para que profissionais de dados possam realizar análises ou até mesmo você. A arquitetura proposta é dividida em duas etapas: transacional, no **Telegram**, onde os dados são produzidos, e analítica, na **Amazon Web Services (AWS)**, onde os dados são analisados.


Dentro dessa estrutura, uma atividade analítica de interesse é a de realizar a análise exploratória de dados enviadas a um **chatbot** para responder perguntas como:


1. Qual o horário que os usuários mais acionam o bot?
2. Qual o problema ou dúvida mais frequente?
3. O bot está conseguindo resolver os problemas ou esclarecer as dúvidas?
4. Etc.

> Uma ilustração de como é a estrutura do chatbot:

<img src="https://github.com/Tobias-Melo/pipeline-telegram/blob/342b475ef25609827e9b11c7312b0c2c7bed8b7d/estrutura_pipeline.png?raw=true" alt="arquitetura-do-projeto">

* **Telegram**

O Telegram representa a fonte de dados transacionais. Mensagens enviadas por usuários em um grupo são capturadas por um bot e redirecionadas via webhook do backend do aplicativo para um endpoint (endereço web que aceita requisições HTTP) exposto pelo AWS API Gateway. As mensagens trafegam no corpo ou payload da requisição.

* **AWS | Ingestão**

Uma requisição HTTP com o conteúdo da mensagem em seu payload é recebia pelo AWS API Gateway que, por sua vez, as redireciona para o AWS Lambda, servindo assim como seu gatilho. Já o AWS Lambda recebe o payload da requisição em seu parâmetro event, salva o conteúdo em um arquivo no formato JSON (original, mesmo que o payload) e o armazena no AWS S3 particionado por dia.

* **AWS | ETL**

Uma vez ao dia, o AWS Event Bridge aciona o AWS Lambda que processa todas as
mensagens do dia anterior (atraso de um dia ou D-1), denormaliza o dado semi-estruturado típico de arquivos no formato JSON, salva o conteúdo processado em um arquivo no formato Apache Parquet e o armazena no AWS S3 particionado por dia.


* **AWS | Apresentação**

Por fim, uma tabela do AWS Athena é apontada para o bucket do AWS S3 que armazena o dado processado: denormalizado, particionado e orientado a coluna. Profissionais de dados ou você podem então executar consultas analíticas (agregações, ordenações, etc.) na tabela utilizando o SQL para a extração de insights.

<h2><strong>2.1. Telegram</strong></h2>

O **Telegram** representa a fonte transacional de dados do nosso *pipeline* de dados. A partir dele, criaremos um grupo e adicionaremos um **bot**. O *bot* então captará **todas** as mensagens enviadas no grupo (videos, mensagens, imagens...). As mensagens pode ser acessadas através da API (*application programming interface*) de *bots* dos **Telegram** (documentação neste [link](https://core.telegram.org/bots/api)).

> Para criar uma conta no Telegram é bem simples, basta fazer o download do aplicativo na loja de aplicativos do seu *smartphone*. Uma vez criada, acesse sua conta através da versão *web* da plataforma de mensagens neste [link](https://web.telegram.org).

<h3 id="arquitetura"><strong>2.1.1. Bot</strong></h3>

A criação do Bot seguem estes passos:

1. Abra o *chat* com o `BotFather`;
1. Digite `/newbot`;
1. Digite o nome do *bot*;
1. Digite o nome de usuário do *bot* (precisa terminar com sufixo `_bot`);
1. Salve o `token` de acesso a API HTTP em um <font color='red'>local seguro</font>.

Caso queira conferir sey *token* novamente basta:

1. Abrir o *chat* com o `BotFather`;
1. Digite `/mybots`;
1. Selecione o *bot* pelo seu nome de usuário;
1. Selecione `API Token`.

Enfim, vamos **ativar** o *bot*:

1. Abra o *chat* com o *bot*;
1. Selecione `start`.

<h3 id="arquitetura"><strong>2.1.2. Grupo</strong></h3>

Passo a passo para criar um novo grupo:

1. Aperte o botão com o ícone de um lápis;
1. Selecione `New Group`;
1. Busque e selecione o *bot* recém criado pelo seu nome;
1. Aperte o botão com o ícone de uma seta;
1. Digite o nome do grupo.

Com o grupo criado, vamos adicionar o *bot* como administrador para que ele possa receber todas as mensagens do grupo. Uma outra opção seria desabilitar o seu modo de privacidade.

1. Abra o *chat* do grupo recém criado;
1. Abra o perfil do grupo;
1. Aperte o botão com o ícone de um lápis;
1. No campo de descrição do grupo escreva: **Atenção, todas as mensagens são armazenadas pelo *bot* do grupo**;
1. Selecione Administrators;
1. Aperte o botão com o ícone de um usuário;
1. Selecione o *bot*.
1. Aperte o botão com o ícone de um *check*.

Por fim, vamos configurar o *bot* para que ele não possa ser adicionado a outros grupos.

1. Abra o *chat* com o `BotFather`;
1. Digite `/mybots`;
1. Selecione o *bot* pelo seu nome de usuário;
1. Selecione `Bot Settings`;
1. Selecione `Allow Groups?`;
1. Selecione `Turn groups off`.

Com tudo pronto, envie algumas mensagens no grupo.

<h2><strong>2.2. Bot API</strong></h2>

As mensagens captadas por um *bot* podem ser acessadas via API. A única informação necessária é o `token` de acesso fornecido pelo `BotFather` na criação do *bot*.

> **Nota:** A documentação completa da API pode ser encontrada neste [link](https://core.telegram.org/bots/api)

In [ ]:
from getpass import getpass

token = getpass()

··········


In [ ]:
import json

import requests

base_url = f'https://api.telegram.org/bot{token}'

* **getMe**

O método `getMe` retorna informações sobre o *bot*.

In [ ]:
response = requests.get(url=f'{base_url}/getMe')

print(json.dumps(json.loads(response.text), indent=2))

{
  "ok": true,
  "result": {
    "id": 5769656381,
    "is_bot": true,
    "first_name": "ebac_bot",
    "username": "pjt_ebac_bot",
    "can_join_groups": false,
    "can_read_all_group_messages": false,
    "supports_inline_queries": false
  }
}


> Perceba que "first_name" é o nome do *bot* e "username" o nome do usuário dado na criação do *bot*.

 - **getUpdates**

O método `getMe` retorna as mensagens captadas pelo *bot*.

In [ ]:
response = requests.get(url=f'{base_url}/getUpdates')

print(json.dumps(json.loads(response.text), indent=2))

{
  "ok": true,
  "result": [
    {
      "update_id": 993973584,
      "message": {
        "message_id": 59,
        "from": {
          "id": 1939175395,
          "is_bot": false,
          "first_name": "Tobias",
          "language_code": "pt-br"
        },
        "chat": {
          "id": -885151066,
          "title": "BOT Child EBAC",
          "type": "group",
          "all_members_are_administrators": true
        },
        "date": 1673917832,
        "text": "Ol\u00e1, mundo!"
      }
    }
  ]
}


<h1 id="dados"><strong>3. Dados</strong></h1>

Antes de avançar para etapa analítica, vamos trabalhar na manipulação dos dados de mensagens do **Telegram**. 

<h2><strong>3.1. Mensagem</strong></h2>

Uma mensagem recuperada via API é um dado semi-estruturado no formato JSON com algumas chaves mandatórias e diversas chaves opcionais, estas últimas presentes (ou não) dependendo do tipo da mensagem. Por exemplo, mensagens de texto apresentam a chave `text` enquanto mensagens de áudio apresentam a chave `audio`. Neste projeto traremos todos os tipos, ou seja, vamos ingerir as chaves mandatórias e as chaves do tipo.

> **Nota**: A lista completa das chaves disponíveis pode ser encontrada na documentação neste [link](https://core.telegram.org/bots/api#message).

 - Exemplo:

In [ ]:
{
    "update_id": 123,
    "message": {
        "message_id": 1,
        "from": {
            "id": 321,
            "is_bot": false,
            "first_name": "Tobias"
        },
        "chat": {
            "id": -789,
            "type": "group"
        },
        "date": 1640995200,
        "text": "Ola, mundo!"
    }
}

 - Descrição:

| chave | tipo valor | opcional | descrição |
| -- | -- | -- | -- |
| updated_id | int | não | id da mensagem enviada ao **bot** |
| message_id | int | não | id da mensagem enviada ao grupo |
| from_id | int | sim | id do usuário que enviou a mensagem |
| from_is_bot | bool | sim | se o usuário que enviou a mensagem é um **bot** |
| from_first_name | str | sim | primeiro nome do usário que enviou a mensagem |
| chat_id | int | não | id do *chat* em que a mensagem foi enviada |
| chat_type | str | não | tipo do *chat*: private, group, supergroup ou channel |
| date | int | não | data de envio da mensagem no formato unix |
| text | str | sim | texto da mensagem |

<h2><strong>3.2. Ingestão</strong></h2>

A etapa de **ingestão** é responsável, como seu o próprio nome diz, pela ingestão dos dados transacionais em ambientes analíticos. De maneira geral, o dado ingerido é persistido no formato mais próximo do original, ou seja, nenhuma transformação é realizada em seu conteúdo ou estrutura (*schema*). Como exemplo, dados de uma API *web* que segue o formato REST (*representational state transfer*) são entregues, logo, persistidos, no formato JSON.

> Persistir os dados em seu formato original trás muitas vantagens, como a possibilidade de reprocessamento.

Pode ser conduzida de duas formas:

 - **Batch**: blocos de dados são ingeridos em uma frequência bem definida, geralmente na escala de horas ou dias;
 - **Streaming**: dados são ingeridos conforme são produzidos e disponibilizados.

No projeto, as mensagens capturadas pelo *bot* podem ser ingeridas através da API *web* de *bots* do **Telegram**, portanto são fornecidos no formato JSON. Como o **Telegram** retem mensagens por apenas 24h em seus servidores, a ingestão via **streaming** é a mais indicada. Para que seja possível esse tipo de **ingestão** seja possível, vamos utilizar um *webhook* (gancho *web*), ou seja, vamos redirecionar as mensagens automaticamente para outra API *web*.

Sendo assim, precisamos de um serviço da AWS que forneça um API *web* para receber os dados redirecionados, o `AWS API Gateway` (documentação neste [link](https://docs.aws.amazon.com/pt_br/apigateway/latest/developerguide/welcome.html)). Dentre suas diversas funcionalidades, o `AWS API Gateway` permite o redirecionamento do dado recebido para outros serviços da AWS. Logo, vamos conecta-lo ao `AWS Lambda`, que pode sua vez, irá armazenar o dado em seu formato original (JSON) em um *bucket* do `AWS S3`.

Seguiremos esses passos:
 - Criar um *bucket* no `AWS S3`;
 - Criar uma função no `AWS Lambda`;
 - Criar uma API *web* no `AWS API Gateway`;
 - Configurar o *webhook* da API de *bots* do **Telegram**. 

> Como os passos são bem detalhados e o objetivo desse projeto é mostrar a excecução do bot, passaremos brevemente por cada passo!

<h3><strong>3.2.1. AWS S3</strong></h3>

Na etapa de **ingestão**, o `AWS S3` tem a função de passivamente armazenar as mensagens captadas pelo *bot* do **Telegram** no seu formato original: JSON. Para tanto, basta a criação de um *bucket*.

> **Curiosidade**: um `data lake` é o nome dado a um repositório de um grande volume dados. É organizado em zonas que armazenam replicadas dos dados em diferentes níveis de processamento. A nomenclatura das zonas varia, contudo, as mais comuns são: *raw* e *enriched* ou *bronze*, *silver* e *gold*.

<h3><strong>3.2.2. AWS Lambda</strong></h3>

---

OS CÓDIGOS UTILIZADOS NO `LAMBDA` ESTÃO DISPONÍVEIS NESTE [LINK](https://github.com/Tobias-Melo/pipeline-telegram).

---

Na etapa de **ingestão**, o `AWS Lambda` tem a função de ativamente persistir as mensagens captadas pelo *bot* do **Telegram** em um *bucket* do `AWS S3`. Para tanto vamos criar uma função que opera da seguinte forma:

 - Recebe a mensagem no parâmetro `event`;
 - Verifica se a mensagem tem origem no grupo do **Telegram** correto;
 - Persiste a mensagem no formato JSON no *bucket* do `AWS S3`;
 - Retorna uma mensagem de sucesso (código de retorno HTTP igual a 200) a API de *bots* do **Telegram**.

Para que os códigos funcionem corretamente, é preciso configurar os seguintes itens:
  - Variáveis de ambiente;
  - Permissões (`AWS IAM`).

<h3><strong>3.2.3. AWS API Gateway</strong></h3>

Na etapa de **ingestão**, o `AWS API Gateway` tem a função de receber as mensagens captadas pelo *bot* do **Telegram**, enviadas via *webhook*, e iniciar uma função do `AWS Lambda`, passando o conteúdo da mensagem no seu parâmetro *event*. Para tanto vamos criar uma API e configurá-la como gatilho da função do `AWS Lambda`:

 - Acesse o serviço e selecione: *Create API* -> *REST API*;
 - Insira um nome;
 - Selecione: *Actions* -> *Create Method* -> *POST*;
 - Na tela de *setup*: 
  - Selecione *Integration type* igual a *Lambda Function*;
  - Habilite o *Use Lambda Proxy integration*;
  - Busque pelo nome a função do `AWS Lambda`.

> Lembre-se de testar a integração com o `AWS Lambda` através da ferramenta de testes do serviço. 

Por fim, vamos fazer a implantação da API e obter o seu endereço *web*.

 - Selecione: *Actions* -> *Deploy API*;
 - Selecione: *New Stage* para *Deployment stage*;
 - Adicione *dev* como `Stage name`.

> **Importante**: Guarde a `url` gerada.

<h3><strong>3.2.4. Telegram</strong></h3>

Vamos configurar o *webhook* para redirecionar as mensagens para a `url` do `AWS API Gateway`.

* setWebhook

O método `setWebhook` configura o redirecionamento das mensagens captadas pelo *bot* para o endereço *web* do paramametro `url`.

> **Nota**: os métodos `getUpdates` e `setWebhook` são mutualmente exclusivos, ou seja, enquanto o *webhook* estiver ativo, o método `getUpdates` não funcionará. Para desativar o *webhook*, basta utilizar o método `deleteWebhook`.

In [ ]:
response = requests.get(url=f'{base_url}/setWebhook?url={aws_api_gateway_url}')

print(json.dumps(json.loads(response.text), indent=2))

Como o meu Webhook já está ativo, o código acima retornará algo como:
```
{
  "ok": true,
  "result": true,
  "description": "Webhook was set"
}
```



 - **getWebhookInfo**

O método `getWebhookInfo` retorna as informações sobre o *webhook* configurado.

In [ ]:
response = requests.get(url=f'{base_url}/getWebhookInfo')

print(json.dumps(json.loads(response.text), indent=2))

Resultado:


```
{
  "ok": true,
  "result": {
    "url": url_do_seu_gateway,
    "has_custom_certificate": false,
    "pending_update_count": 0,
    "max_connections": 40,
    "ip_address": "********"
  }
}
```



<h2><strong>3.3. ETL</strong></h2>

A etapa de **extração, transformação e carregamento** (do inglês *extraction, transformation and load* ou **ETL**) é uma etapa abrangente responsável pela manipulação dos dados ingeridos de sistemas transacionais, ou seja, já persistidos em camadas cruas ou *raw* de sistemas analíticos. Os processos conduzidos nesta etapa variam bastante de acordo com a área da empresa, do volume/variedade/velocidade do dado consumido, etc. Contudo, em geral, o dado cru ingerido passa por um processo recorrente de *data wrangling* onde o dado é limpo, deduplicado, etc. e persistido com técnicas de particionamento, orientação a coluna e compressão. Por fim, o dado processado está pronto para ser analisado por profissionais de dados.

No projeto, as mensagens de um único dia, persistidas na camada cru, serão compactas em um único arquivo, orientado a coluna e comprimido, que será persistido em uma camada enriquecida. Além disso, durante este processo, o dado também passará por etapas de *data wrangling*.

Para isso, vamos utilizar uma função do `AWS Lambda` como motor de processamento e um *bucket* do `AWS S3` como camada enriquecida para a persistência do dado processado. Para garantir a recorrência, vamos configurar uma regra do `AWS Event Bridge` como gatilho diáro da função.

<h3><strong>3.3.1. AWS S3</strong></h3>

Na etapa de **ETL**, o `AWS S3` tem a função de passivamente armazenar as mensagens processadas de um dia em um único arquivo no formato Parquet. Para tanto, basta a criação de um *bucket*.

<h3><strong>3.3.2. AWS Lambda</strong></h3>

Na etapa de **ETL**, o `AWS Lambda` tem a função de ativamente processar as mensagens captadas pelo *bot* do **Telegram**, persistidas na camada cru no *bucket* do `AWS S3`, e persisti-las na camada enriquecida, também em um *bucket* do `AWS S3`. Logo, vamos criar uma função que opera da seguinte forma:

 - Lista todos os arquivos JSON de uma única participação da camada crua de um *bucket* do `AWS S3`;
 - Para cada arquivo listado:
  - Faz o *download* do arquivo e carrega o conteúdo da mensagem;
  - Executa uma função de *data wrangling*;
  - Cria uma tabela do PyArrow e a contatena com as demais.
 - Persiste a tabela no formato Parquet na camada enriquecida em um *bucket* do `AWS S3`.

> **Nota**: O fato de utilizarmos duas camadas de armazenamento e processamento, permite que possamos reprocessar os dados crus de diversas maneiras, quantas vezes forem preciso.
**Atente-se** ao fato de que a função processa as mensagens do dia anterior (D-1). 

---

OS CÓDIGOS UTILIZADOS NO `LAMBDA` ESTÃO DISPONÍVEIS NESTE [LINK](https://github.com/Tobias-Melo/pipeline-telegram).

---

Para que os códigos funcionem corretamente, é preciso configurar os seguintes itens:
  - Variáveis de ambiente;
  - Permissões (`AWS IAM`);
  - Recursos;
  - Camadas.

Vamos detalhar os *Recursos* e *Camadas*.

 - **Recursos**

O *timeout* padrão de funcões do `AWS Lambda` é de 3 segundos. Para a função, vamos aumentar o tempo para 5 minutos, principalmente para lidar com o IO (*input/output*) de arquivos do `AWS S3`.

 - **Camadas**

Por fim, note que o código da função utiliza o pacote Python PyArrow. Contudo, o ambiente padrão do `AWS Lambda` possui poucos pacotes externos instalado, como o pacote Python boto3, logo o PyArrow não será encontrado e a execução da função falhará. Existem algumas formas de adicionar pacotes externos no ambiente de execução do AWS Lambda, um deles é a criação de camadas ou *layers*, onde podemos fazer o *upload* dos pacotes Python direto na plataforma ou através de um *bucket* do `AWS S3`. Vamos então seguir com a última opção, onde teremos que:

 - Criar um *bucket* no `AWS S3`;
 - Fazer o *upload* do código do pacote Python do PyArrow (*download* neste [link](https://github.com/awslabs/aws-data-wrangler/releases));
 - Criar *layer* e conectar na função.

<h3><strong>3.3.2. AWS Event Bridge</strong></h3>

Na etapa de **ETL**, o `AWS Event Bridge` tem a função de ativar diariamente a função de **ETL** do `AWS Lambda`, funcionando assim como um *scheduler*.

<h1 id="analise"><strong>5. Análise Exploratória de Dados</strong></h1>

Antes de iniciarmos a análise sobre os dados capturados pelo *bot* veremos como criar uma tabela para consultas no **AWS Athena**.

<h3><strong>5.1. AWS Athena</strong></h3>

O `AWS Athena` tem função de entregar o dados através de uma interface SQL para os usuários do sistema analítico. Para criar a interface, basta criar uma tabela externa sobre o dado armazenado na camada mais refinada da arquitetura, a camada enriquecida.

```sql
CREATE EXTERNAL TABLE `telegram`(
  `message_id` bigint, 
  `user_id` bigint, 
  `user_is_bot` boolean, 
  `user_first_name` string, 
  `chat_id` bigint, 
  `chat_type` string, 
  `date` bigint,
  `message_type` string,
  `message_content` string,
  `context_date` date)
PARTITIONED BY ( 
  `context_date` date)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://<bucket-enriquecido>/'
```

> **Importante**: Toda vez que uma nova partição é adicionada ao repositório de dados, é necessário informar o `AWS Athena` para que a ela esteja disponível via SQL. Para isso, use o comando SQL `MSCK REPAIR TABLE <nome-tabela>` para todas as partições (mais caro) ou `ALTER TABLE <nome-tabela> ADD PARTITION <coluna-partição> = <valor-partição>` para uma única partição (mais barato), documentação neste [link](https://docs.aws.amazon.com/athena/latest/ug/alter-table-add-partition.html)). Nos códigos disponibilizados não é necessário realizar essa tarefa, pois, já existe esse processo dentro da função de enriquecimento.

* Estrutura da tabela:

<img src="https://github.com/Tobias-Melo/pipeline-telegram/blob/7a22fdc2b9d201c16186299b03c390141de0f4c9/estrutura_tabela.PNG?raw=true">

<h3><strong>5.2. Analytics</strong></h3>

Com o dado disponível, usuário podem executar as mais variadas consultas analíticas. Seguem alguns exemplos:

- Quantidade de mensagens por dia.

```sql
SELECT 
  context_date, 
  count(1) AS "message_amount" 
FROM "telegram" 
GROUP BY context_date 
ORDER BY context_date DESC
```

- Quantidade de mensagens por usuário por dia.

```sql
SELECT 
  user_id, 
  user_first_name, 
  context_date, 
  count(1) AS "message_amount" 
FROM "telegram" 
GROUP BY 
  user_id, 
  user_first_name, 
  context_date 
ORDER BY context_date DESC
```

- Quantidade de mensagens por tipo de mensagem por dia.

```sql
SELECT 
    message_type, 
    count(1) AS "message_amount" 
FROM "telegram" 
GROUP BY 
    message_type
ORDER BY message_amount DESC
```

- Média do tamanho das mensagens por usuário por dia.

```sql
SELECT 
  user_id, 
  user_first_name, 
  context_date,
  CAST(AVG(length(message_content)) AS INT) AS "average_message_length" 
FROM "telegram" 
GROUP BY 
  user_id, 
  user_first_name, 
  context_date 
ORDER BY context_date DESC
```

- Quantidade de mensagens por hora por dia da semana por número da semana.

```sql
WITH 
parsed_date_cte AS (
    SELECT 
        *, 
        CAST(date_format(from_unixtime("date"),'%Y-%m-%d %H:%i:%s') AS timestamp) AS parsed_date
    FROM "telegram" 
),
hour_week_cte AS (
    SELECT
        *,
        EXTRACT(hour FROM parsed_date) AS parsed_date_hour,
        EXTRACT(dow FROM parsed_date) AS parsed_date_weekday,
        EXTRACT(week FROM parsed_date) AS parsed_date_weeknum
    FROM parsed_date_cte
)
SELECT
    parsed_date_hour,
    parsed_date_weekday,
    parsed_date_weeknum,
    count(1) AS "message_amount" 
FROM hour_week_cte
GROUP BY
    parsed_date_hour,
    parsed_date_weekday,
    parsed_date_weeknum
ORDER BY
    parsed_date_weeknum,
    parsed_date_weekday
```

Chegamos ao fim.

Essa é apenas a ponta do iceberg com as possibilidades do BotFather. Espero que desfrutem do projeto e possam aplicá-lo dentro da sua possilidade. Até breve!